In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

# --- Parámetros ---
roi = gpd.read_file('./data/area_mobility_workshop')
# Diccionario de categorías de OSM para cada perfil
profiles_pois = {
    "elderly": {
        "residential": ["residential"],
        "grocery_store": ["supermarket", "convenience", "marketplace"],
        "plaza": ["square", "park"]
    },
    "student": {
        "university": ["university", "college", "school"],
        "pub": ["pub", "bar", "nightclub"],
        "gym": ["gym", "fitness_centre", "sports_centre"]
    },
    "office_worker": {
        "office": ["office", "commercial"],
        "restaurant": ["restaurant", "cafe", "fast_food"],
        "residential": ["residential"]
    },
    "tourist": {
        "tourist_places": ["museum", "art_gallery", "viewpoint", "attraction", "monument", "historic", "tourism"]
    },
    "families": {
        "park": ["park", "playground", "garden"],
        "school": ["school", "kindergarten"],
        "residential": ["residential"]
    },
    "shop_owner": {
        "residential": ["residential"],
        "market": ["marketplace", "supermarket", "convenience"],
        "storefront": ["shop"]
    }
}

In [2]:
import geopandas as gpd

# Cargar tu archivo exportado previamente
gdf = gpd.read_file("./data/pois.gpkg", layer="pois")

# Definir categorías y reglas de mapeo
category_rules = {
    "storefront": {"shop": True},
    "university": {"amenity": "university"},
    "cafe": {"amenity": "cafe"},
    "grocery_store": {"shop": ["supermarket", "convenience"], "amenity": "marketplace"},
    "restaurant": {"amenity": "restaurant"},
    "market": {"amenity": "marketplace"},
    "residential": {"building": "residential", "landuse": "residential"},
    "pub": {"amenity": "pub"},
    "tourist_places": {"tourism": ["attraction", "museum", "viewpoint"]},
    "park": {"leisure": "park"},
    "school": {"amenity": "school"},
    "office": {"office": True},
    "plaza": {"leisure": "plaza", "amenity": "town_square"},
    "gym": {"amenity": "gym", "leisure": "fitness_centre", "sport": ["fitness", "gymnastics"]},
}

# Función para asignar categoría
def assign_category(row):
    for cat, rules in category_rules.items():
        for key, values in rules.items():
            if key not in row or pd.isna(row.get(key)):
                continue
            val = row.get(key)
            if values is True:  # Cualquier valor sirve
                return cat
            if isinstance(values, list) and val in values:
                return cat
            if val == values:
                return cat
    return None

# Crear nueva columna
gdf["category"] = gdf.apply(assign_category, axis=1)

# Si alguna quedó sin categoría, usa la columna main_category original como fallback
gdf["category"] = gdf["category"].fillna(gdf.get("main_category"))

# Exportar resultado
gdf.to_file("./data/pois_categorizados.gpkg", layer="pois", driver="GPKG")
gdf.to_file("./data/pois_categorizados.geojson", driver="GeoJSON")

print(f"✅ Re-etiquetación lista. Total filas: {len(gdf)}")
print(gdf[["main_category", "category", "amenity", "shop", "leisure", "tourism", "sport"]].head())


✅ Re-etiquetación lista. Total filas: 831
  main_category    category amenity         shop leisure tourism sport
0    storefront  storefront    None          art    None    None  None
1    storefront  storefront    None       pastry    None    None  None
2    storefront  storefront    None  convenience    None    None  None
3    storefront  storefront    None  convenience    None    None  None
4    storefront  storefront    None  supermarket    None    None  None


In [3]:
gdf_points = gdf[gdf.geometry.type == 'Point'].copy()
gdf_points_filtered = gdf_points[gdf_points['category'].notna()].copy()
gdf_points_filtered = gpd.overlay(gdf_points_filtered, roi, how='intersection')

In [4]:
gdf_polygons = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])].copy()
gdf_polygons_filtered = gdf_polygons[gdf_polygons['category'].notna()].copy()
gdf_polygons_filtered = gpd.overlay(gdf_polygons_filtered, roi, how='intersection')

In [5]:
gdf_filtered = pd.concat([gdf_points_filtered, gdf_polygons_filtered], ignore_index=True)
gdf_filtered = gpd.GeoDataFrame(gdf_filtered, geometry='geometry', crs=gdf.crs)

In [6]:
gdf_filtered.to_parquet("./data/pois_categorizados_filtrados.parquet", index=False)

In [8]:
gdf_filtered['category'].value_counts()

category
storefront        451
office            128
restaurant         74
residential        45
cafe               38
school             26
pub                23
university         13
park               13
gym                 9
tourist_places      7
grocery_store       4
Name: count, dtype: int64